## 1. 重塑层次化索引
`stack()` 将数据的列转为行，返回一个有层次化索引的DataFrame或Series  
- `level` 指定将哪层列转换为行，默认最内层即-1，可以使用编号或者name属性
- `dropna` 布尔值，是否过滤缺失值，默认True

`unstack()` 将数据的行转为列
- `level` 指定将哪层行转换为列，默认最内层即-1，可以使用编号或者name属性
- `fill_value` 使用指定值来填充缺失值
  
`stack()`和`unstack()`之间的转换是可逆的。  
若转换的行或列是表格的唯一行或列，转换得到的结果就是一个Series

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame(np.random.randint(10, size=6).reshape(2, 3), 
                    index=pd.Index(['Tom','Jerry'],name='Name'), 
                    columns=pd.Index(['English', 'Math', 'Chinese'], name='subject'))
data.loc['Tom','Math']=np.nan
data

subject,English,Math,Chinese
Name,,,
Tom,8,NaN,0
Jerry,7,3.0,0


In [3]:
# 默认将最内层的列转为最内层行索引，并默认过滤缺失值
score = data.stack()
score

Name   subject
Tom    English    8.0
       Chinese    0.0
Jerry  English    7.0
       Math       3.0
       Chinese    0.0
dtype: float64

In [4]:
# 使用unstack默认将最内层的行转换为列
score.unstack()

subject,English,Math,Chinese
Name,,,
Tom,8.0,NaN,0.0
Jerry,7.0,3.0,0.0


In [5]:
# 指定将最外层行转换为列，并将缺失值填充10
# 也可以使用行索引的name属性'Name'指定需要转换的行索引
score.unstack(0,fill_value=10)

Name,Tom,Jerry
subject,,
English,8.0,7.0
Math,10.0,3.0
Chinese,0.0,0.0


## 2. 轴向旋转
有这样一种表格，其中的某些列只有固定的几个值，这些值也可以作为列来表示，使得表格结构更加清晰。  
例如下面这个表格，其中科目和考试类型都可以转换成列，因此可以使用`pivot`和`pivot_table`进行转换。  
`pivot(index,columns,values)` 指定哪个列作为index，哪个列的值作为新的列，哪个列的值作为新表格的数据值，不指定数据值列的话剩余的列都作为数据值，得到层次化列的表  
`pivot_table(index,columns,values,aggfunc)` 指定哪（几）个列作为index，哪（几）个列的值作为新的列，哪（几）个列的值经过`aggfunc`指定方法处理后作为新表格的数据值  

In [6]:
data=pd.read_csv('doc/pivot-example.csv',encoding='GBK')
data

,姓名,科目,考试类型,考试成绩,平时成绩
0,张三,英语,期中,80,75
1,李四,英语,期中,77,72
2,王二,英语,期中,90,85
3,张三,数学,期中,88,83
4,李四,数学,期中,60,55
5,王二,数学,期中,72,67
6,张三,语文,期中,85,80
7,李四,语文,期中,92,87
8,王二,语文,期中,68,63
9,张三,英语,期末,79,74


In [7]:
# 先取部分数据进行pivot处理
d=data.loc[:8,['姓名','科目','考试成绩','平时成绩']]
d

,姓名,科目,考试成绩,平时成绩
0,张三,英语,80,75
1,李四,英语,77,72
2,王二,英语,90,85
3,张三,数学,88,83
4,李四,数学,60,55
5,王二,数学,72,67
6,张三,语文,85,80
7,李四,语文,92,87
8,王二,语文,68,63


In [8]:
# 将姓名作为行，科目内容作为列，其余列为数据值
# 由于数据值有两列，因此生成层次化数据值
d.pivot('姓名','科目')

考试成绩         平时成绩        
科目   数学  英语  语文   数学  英语  语文
姓名                          
张三   88  80  85   83  75  80
李四   60  77  92   55  72  87
王二   72  90  68   67  85  63

In [9]:
# 将姓名作为行，将考试类型作为外层列，科目作为内层列
# 此方法等价于data.set_index(['姓名','科目','考试类型']).unstack(['考试类型','科目'])
data.pivot_table(index='姓名',columns=['考试类型','科目'])

平时成绩                     考试成绩                    
考试类型   期中          期末           期中          期末        
科目     数学  英语  语文  数学  英语  语文   数学  英语  语文  数学  英语  语文
姓名                                                    
张三     83  75  80  72  74  85   88  80  85  77  79  90
李四     55  72  87  59  79  69   60  77  92  64  84  74
王二     67  85  63  78  87  90   72  90  68  83  92  95

In [10]:
# 将姓名作为行，科目内容作为列，考试成绩作为数据值
# 此时两个成绩还分为期中和期末两种类型，使用aggfunc指定方法对成绩进行处理
# 这里我们取两种考试类型中成绩最高的数据
data.pivot_table(index='姓名',columns='科目',values=['考试成绩','平时成绩'],aggfunc=np.max)

平时成绩         考试成绩        
科目   数学  英语  语文   数学  英语  语文
姓名                          
张三   83  75  85   88  80  90
李四   59  79  87   64  84  92
王二   78  87  90   83  92  95

`melt(df,id_vars,value_vars)` pivot的逆运算，将多个列标签合并为一个以columns的name属性命名的新列，产生一个新的表  
参数  
- `id_vars` 用作标识的列
- `value_vars` 要合并的列，列的值作为数据值

In [11]:
score=d.pivot('姓名','科目','考试成绩')
score.reset_index(inplace=True) # 将索引数据移回列
score

科目,姓名,数学,英语,语文
0,张三,88,80,85
1,李四,60,77,92
2,王二,72,90,68


In [12]:
# 使用姓名作为标识，其余列合并
pd.melt(score,['姓名'])

,姓名,科目,value
0,张三,数学,88
1,李四,数学,60
2,王二,数学,72
3,张三,英语,80
4,李四,英语,77
5,王二,英语,90
6,张三,语文,85
7,李四,语文,92
8,王二,语文,68


In [13]:
# 指定某几个列合并
pd.melt(score,id_vars='姓名',value_vars=['数学','语文'])

,姓名,科目,value
0,张三,数学,88
1,李四,数学,60
2,王二,数学,72
3,张三,语文,85
4,李四,语文,92
5,王二,语文,68


In [14]:
# 也可以不指定标识
pd.melt(score)

,科目,value
0,姓名,张三
1,姓名,李四
2,姓名,王二
3,数学,88
4,数学,60
5,数学,72
6,英语,80
7,英语,77
8,英语,90
9,语文,85
